In [1]:
import pandas
import numpy as np
from sklearn.decomposition import PCA

In [3]:
#1 PCA
data = pandas.read_csv('close_prices.csv')

In [4]:
X = data[data.columns[1:]]

In [5]:
pca = PCA(n_components = 10)
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [6]:
print(pca.explained_variance_ratio_)

[ 0.73897118  0.11007169  0.04995088  0.0287492   0.02215448  0.01931577
  0.00674853  0.00614091  0.00320594  0.00305611]


In [7]:
s = 0
for i in range(len(pca.explained_variance_ratio_)):
    s += pca.explained_variance_ratio_[i]
    if (s > 0.9):
        print(i+1)
        break
with open('week4_1', 'w') as f:
    f.write(str(i+1))

4


In [8]:
#2
index = pandas.read_csv('djia_index.csv')

In [9]:
dji = index['^DJI']

In [10]:
with open('week4_2', 'w') as f:
    f.write(str(round(np.corrcoef(pca.transform(X).T[0],dji)[0][1],2)))

In [16]:
#3
with open('week4_3', 'w') as f:
    f.write(data.columns[pca.components_[0].argmax()+1])

In [2]:
#4 линейная регрессия
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack, coo_matrix
from sklearn.linear_model import Ridge

In [6]:
data_train = pandas.read_csv('salary-train.csv')
data_test = pandas.read_csv('salary-test-mini.csv')

In [12]:
def clear_text(pd):
    return pandas.DataFrame(pd.str.lower().str.replace('[^a-z0-9]', ' ', regex = True))

In [13]:
text_train = clear_text(data_train['FullDescription'])
text_test = clear_text(data_test['FullDescription'])

In [83]:
vectorizer = TfidfVectorizer(min_df = 5)
X_train_text = vectorizer.fit_transform(text_train.FullDescription)
X_test_text = vectorizer.transform(text_test.FullDescription)

In [38]:
data_train['LocationNormalized'].fillna('nan', inplace = True)
data_train['ContractTime'].fillna('nan', inplace = True)

In [84]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('record'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('record'))

In [86]:
X_train = hstack([X_train_text,X_train_categ])
X_test = hstack([X_test_text, X_test_categ])

In [97]:
y_train = data_train['SalaryNormalized']

In [98]:
ridge = Ridge(alpha=1, random_state=241)
ridge.fit(X_train, y_train)
y_test = ridge.predict(X_test)
print(y_test)

[ 56555.61500155  37188.32442618]


In [100]:
with open('week4_4', 'w') as f:
    f.write(str(round(y_test[0],2)) + ' ' + str(round(y_test[1],2)))